# Ленивые вычисления

Что такое, зачем нужны?

![alt text](http://atkritka.com/upload/iblock/18e/atkritka_1341782872_334.jpg)

## Шаг 1: Осознание проблемы

Допустим, есть два файла заданного формата. Нужно считать эти файла и записать их содержимое в третий, причем над каждой строкой из файла произвести какую-либо операцию (не важно, какую).

Если попытаться сначала считать файлы, будет не очень хорошо. Особенно если файлы большие.

Что делать?

## Шаг 2: Анализ возможных решений

1. открыть файл 1;
2. открыть файл 2;
3. ???
4. PROFIT

![alt text](https://lurkmore.so/images/2/2e/PROFIT.jpg)

## Решение - делать лениво

Концепция **мы ничего не делаем до тех пор, пока нам это действительно сильно не понадобится**.

В нашем случае открываем файлы и читаем по строчке. Обрабатываем по строчке. В ОП не лежит больше одной строчки из файла.

Это в и есть концепция ленивых вычислений.

## Пример на Ruby

Пять первых чисел бесконечной последовательности

In [6]:
(1..Float::INFINITY).lazy.reject{|i| i.odd?}.map{|i| i*i}.first(5)

[4, 16, 36, 64, 100]

Более разумно: берем из последовательности пока выполняется условие. **Так нужно считать с произвольной точностью.**

In [11]:
range = (1..Float::INFINITY).lazy.reject{|i| i.odd?}.map{|i| i*i}.take_while{|i| i < 100}

p range

range.each do |i|
  p i
end

#<Enumerator::Lazy: #<Enumerator::Lazy: #<Enumerator::Lazy: #<Enumerator::Lazy: 1..Infinity>:reject>:map>:take_while>
4
16
36
64


Обратите внимание, что хотя мы и записали выражение в переменную range, оно не вычисляется до тех пор, пока мы не вызвали each. И даже тогда оно вычисляется не целиком!

**Если убрать вызов метода lazy из примеров, то все будет вычисляться в том месте, где описано, то есть сразу**

## Что такое Enumerator?

Это класс, который позволяет итерироваться по коллекции. Вот тут Enumerator используется

In [10]:
[1, 2, 3].each do |x|
  p x
end

1
2
3


[1, 2, 3]

А вот так его можно получить

In [11]:
enum = [1, 2, 3].each
enum.class

Enumerator

In [12]:
p enum.next
enum.next

1


2

In [13]:
enum.next

3

А дальше?

In [14]:
enum.next

StopIteration: iteration reached an end

Enumerator - это отдельный класс, есть еще примесь Enumerable. Enumerator и Enumerable делают по сути одну и ту же работу.

## По умолчанию Enumerator не ленивый

Чтобы это исправить, есть метод lazy. Он говорит интерпретатору о том, что нужно отложить выполнение кода.

## Примеры, где это может пригодиться

### 1

Вычислить сумму ряда $$S = 1 + \sum_{k=1}^\infty 1/{k!}$$

Точное значение - e = 2.718281828...

In [9]:
enum_solve = Enumerator.new do |yielder|
  sum, k_factorial, k = 1, 1, 1
  
  loop do
    yielder.yield sum
    
    k_factorial *= k
    k += 1
    
    sum += 1 / k_factorial.to_f
  end
end

enum_solve.take_while { |sum| (Math::E - sum).abs > 10e-5 }

[1, 2.0, 2.5, 2.6666666666666665, 2.708333333333333, 2.7166666666666663, 2.7180555555555554]

### 2

Сравнить два файла: F и G. Вывести true, если файлы совпадают, и 0 в противном случае.

In [51]:
def equal?(file1, file2)
  result = true
  
  File.open file1, 'r' do |f|
    File.open file2, 'r' do |g|
      f.zip(g).lazy.each do |line_f, line_g|
        unless line_f == line_g
          result = false
          break
        end
      end
    end
  end
  
  result
end

p equal? '2_1', '2_2'
equal? '2_2', '2_2_equal'

false
true


true

### 3

Сформировать программным путем файл F, компоненты которого являются целыми числами. Получить файл G, образованный из файла F исключением повторных вхождений одного и того же числа.

Создаем файл F

In [48]:
File.open 'F', 'w' do |f|
  1488.times do
    f.write (rand() * 1000).to_i.to_s + "\n"
  end
end

1488

Создаем файл G

In [55]:
line_set = Set.new

File.open 'G', 'w' do |g|
  File.open 'F', 'r' do |f|
    f.each_line.lazy.each do |line|
      if line_set.add? line
        g.write line
      end
    end
  end
end

#<File:F (closed)>

### 4

Решить задачу, организовав цикл. Вычислить определенный интеграл $$\int_{\pi/4}^{\pi/3} tg^2(x)dx$$ методом трапеций:
Отрезок интегрирования разбить на n = 20, 30, 40 частей. Точное значение: $$\sqrt{3} - \pi/12 - 1$$

In [1]:
CORRECT = Math.sqrt(3) - Math::PI / 12.0 - 1

0.47025141976972784

In [2]:
def cyclic_solve lower_bound, upper_bound, n_parts
  step = (upper_bound - lower_bound).abs.to_f / n_parts.to_f
  
  prev, cur = lower_bound, lower_bound + step
  
  area = 0
  
  n_parts.times do
    area += step * (Math.tan(cur)**2 + Math.tan(prev)**2) / 2.0
    prev = cur
    cur += step
  end
  
  area
end

:cyclic_solve

In [3]:
cyclic_solve(Math::PI / 4.0, Math::PI / 3.0, 10)

0.47081403105654274

То же самое с помощью Enumerator:

In [ ]:
def enum_solve(n_steps, lower_bound, upper_bound)
  step = (upper_bound - lower_bound).abs.to_f / n_steps.to_f
  prev, cur = lower_bound, lower_bound + step
  area = 0
  
  Enumerator::Lazy.new 1..n_steps do |yielder|
    area += step * (Math.tan(cur)**2 + Math.tan(prev)**2) / 2.0
    prev = cur
    cur += step
    
    yielder.yield area
  end
end

enum_solve(10, Math::PI / 4.0, Math::PI / 3.0).each do |x|
  p x
end

## То же самое, но написанное программистом на Perl

In [31]:
def enum_solve(n_steps, lower_bound, upper_bound)
  step = (upper_bound - lower_bound).abs.to_f / n_steps.to_f
  prev, cur = lower_bound, lower_bound + step
  area = 0
  
  Enumerator::Lazy.new 1..n_steps do |yielder|
    area += step * (Math.tan(cur)**2 + Math.tan(prev)**2) / 2.0
    prev = cur
    cur += step
    
    yielder << area
  end.each {|x| area = x}
  
  area
end

enum_solve(10, Math::PI / 4.0, Math::PI / 3.0)

0.47081403105654274

![](sticker.png)